In [10]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import experimenter
import pandas as pd 
import matplotlib.pyplot as plt 

sns.set_theme()

In [13]:
# Maximum number of replicas that the system can manage.
MAX_REPLICA_COUNT = 100

# Number of experiments
NUMBER_OF_EXPERIMENTS = 5

# Each experiment will be done by this many times for consistency.
EXPERIMENT_PROOF = 5

# Int array of replicas for each experiment
REPLICAS: np.ndarray = experimenter.calculate_number_of_replicas(MAX_REPLICA_COUNT, NUMBER_OF_EXPERIMENTS)

# How much should we wait for the kubernetes api to be stabilized
COOLDOWNS: np.ndarray = 2 * np.ones((NUMBER_OF_EXPERIMENTS,), dtype=int) + (REPLICAS * 0.004).astype(int)

# Namespace of the experimentation
NAMESPACE = "experiments"

# Driver used in kubectl
DRIVER = "minikube"

# Framework
FRAMEWORK = "native"

In [12]:
def draw_replicas(replicas=REPLICAS):
    plt.figure(figsize=(8, 4))
    plt.plot(range(len(replicas)), replicas)
    plt.title("Replica Count by Experiments")
    plt.xlabel("Experiment")
    plt.ylabel("Replicas")
    plt.show()

# draw_replicas()
# print(COOLDOWNS)
# print(REPLICAS)

In [14]:
# Do the experiment
# experiment_with(REPLICAS.tolist(), NAMESPACE, COOLDOWNS.tolist(), EXPERIMENT_PROOF, DRIVER)

experimenter.experiment_with(FRAMEWORK, DRIVER, 1, [1, 1], [1, 1])

Estimated time 2 second(s) will be saved in './exps/20230228_004310.feather'.
Experiment of 1 step(s) with 1 replica(s) in 'native' and 'busybox' image in namespace 'experiments'. Estimated time for the experiment: 1 second(s)
Experiment of 1 step(s) with 1 replica(s) in 'native' and 'busybox' image in namespace 'experiments'. Estimated time for the experiment: 1 second(s)
Done!


In [9]:
KNATIVE_EXPERIMENT_NAME = "exps/20230227_232952.feather"

def process_experiment(filename):
    data_df = pd.read_feather(filename)

    data_df["api_server_creation_delta"] = data_df["event_created_at"] - data_df["before_create_timestamp"]
    data_df["api_server_deletion_delta"] = data_df["event_deleted_at"] - data_df["before_delete_timestamp"]
    
    data_df.to_feather(filename)
    return data_df

knative_minikube_df = process_experiment(KNATIVE_EXPERIMENT_NAME)
knative_minikube_df

,before_create_timestamp,before_firstwait_timestamp,before_delete_timestamp,before_secondwait_timestamp,before_cooldown_timestamp,before_join_timestamp,replicas,framework,error_occured,errors_rised,...,total_steps,cool_period,driver,command,event_created_at,event_deleted_at,event_modified_at,event_replicas_at,api_server_creation_delta,api_server_deletion_delta
0,2023-02-27 23:29:52.732550,2023-02-27 23:29:52.773953,2023-02-27 23:29:55.977492,2023-02-27 23:29:56.005130,2023-02-27 23:29:56.005134,2023-02-27 23:29:58.005220,1,knative,False,"[None, None, None]",...,1,2,minikube,"[/bin/sh, -c, sleep 9999]",2023-02-27 23:29:52.770429,2023-02-27 23:29:56.003805,"[2023-02-27 23:29:52.801613, 2023-02-27 23:29:...","[nan, nan, nan, 1.0]",0 days 00:00:00.037879,0 days 00:00:00.026313
1,2023-02-27 23:29:58.534350,2023-02-27 23:29:58.584932,2023-02-27 23:30:02.131228,2023-02-27 23:30:02.149399,2023-02-27 23:30:02.149401,2023-02-27 23:30:04.149493,1,knative,False,"[None, None, None]",...,1,2,minikube,"[/bin/sh, -c, sleep 9999]",2023-02-27 23:29:58.577167,2023-02-27 23:30:02.149220,"[2023-02-27 23:29:58.605398, 2023-02-27 23:29:...","[nan, nan, nan, 1.0]",0 days 00:00:00.042817,0 days 00:00:00.017992


In [ ]:
# BETTER PROCESS THIS DATA
plt.figure(figsize=(20, 4))
plt.subplot(1, 2, 1)
plt.plot(knative_minikube_df["api_server_deletion_delta"]); plt.title("Creation Times")
plt.subplot(1, 2, 2)
plt.plot(knative_minikube_df["api_server_creation_delta"]); plt.title("Deletion Times")
plt.show()